
     Fast Start Example #5 - RAG with Semantic Query<br>
    This example illustrates the most common RAG retrieval pattern, which is using a semantic query, e.g.,<br>
    a natural language query, as the basis for retrieving relevant text chunks, and then using as<br>
    the context material in a prompt to ask the same question to a LLM.<br>
    In this example, we will show the following:<br>
    1.  Create library and install embeddings (feel free to skip / substitute a library created in an earlier step).<br>
    2.  Ask a general semantic query to the entire library collection.<br>
    3.  Select the most relevant results by document.<br>
    4.  Loop through all of the documents - packaging the context and asking our questions to the LLM.<br>
    NOTE: to use chromadb, you may need to install the python sdk:  pip3 install chromadb.<br>


In [1]:
import os
from llmware.library import Library
from llmware.retrieval import Query
from llmware.setup import Setup
from llmware.status import Status
from llmware.prompts import Prompt
from llmware.configs import LLMWareConfig
from importlib import util
if not util.find_spec("chromadb"):
    print("\nto run this example with chromadb, you need to install the chromadb python sdk:  pip3 install chromadb")

In [3]:
def semantic_rag (library_name, embedding_model_name, llm_model_name):
    """ Illustrates the use of semantic embedding vectors in a RAG workflow
        --self-contained example - will be duplicative with some of the steps taken in other examples """

    # Step 1 - Create library which is the main 'organizing construct' in llmware
    print ("\nupdate: Step 1 - Creating library: {}".format(library_name))
    library = Library().create_new_library(library_name)

    # Step 2 - Pull down the sample files from S3 through the .load_sample_files() command
    #   --note: if you need to refresh the sample files, set 'over_write=True'
    print ("update: Step 2 - Downloading Sample Files")
    sample_files_path = Setup().load_sample_files(over_write=False)
    contracts_path = os.path.join(sample_files_path, "Agreements")

    # Step 3 - point ".add_files" method to the folder of documents that was just created
    #   this method parses all of the documents, text chunks, and captures in MongoDB
    print("update: Step 3 - Parsing and Text Indexing Files")
    library.add_files(input_folder_path=contracts_path, chunk_size=400, max_chunk_size=600,
                      smart_chunking=1)

    # Step 4 - Install the embeddings
    print("\nupdate: Step 4 - Generating Embeddings in {} db - with Model- {}".format(vector_db, embedding_model))
    library.install_new_embedding(embedding_model_name=embedding_model_name, vector_db=vector_db)

    # RAG steps start here ...
    print("\nupdate: Loading model for LLM inference - ", llm_model_name)
    prompter = Prompt().load_model(llm_model_name)
    query = "what is the executive's base annual salary"

    #   key step: run semantic query against the library and get all of the top results
    results = Query(library).semantic_query(query, result_count=50, embedding_distance_threshold=1.0)

    #   if you want to look at 'results', uncomment the two lines below
    #   for i, res in enumerate(results):
    #       print("update: ", i, res["file_source"], res["distance"], res["text"])
    for i, contract in enumerate(os.listdir(contracts_path)):
        qr = []
        if contract != ".DS_Store":
            print("\nContract Name: ", i, contract)

            #   we will look through the list of semantic query results, and pull the top results for each file
            for j, entries in enumerate(results):
                library_fn = entries["file_source"]
                if os.sep in library_fn:
                    # handles difference in windows file formats vs. mac / linux
                    library_fn = library_fn.split(os.sep)[-1]
                if library_fn == contract:
                    print("Top Retrieval: ", j, entries["distance"], entries["text"])
                    qr.append(entries)

            #   we will add the query results to the prompt
            source = prompter.add_source_query_results(query_results=qr)

            #   run the prompt
            response = prompter.prompt_with_source(query, prompt_name="default_with_context", temperature=0.3)

            #   note: prompt_with_resource returns a list of dictionary responses
            #   -- depending upon the size of the source context, it may call the llm several times
            #   -- each dict entry represents 1 call to the LLM
            for resp in response:
                if "llm_response" in resp:
                    print("\nupdate: llm answer - ", resp["llm_response"])

            # start fresh for next document
            prompter.clear_source_materials()
    return 0

In [4]:
if __name__ == "__main__":
    LLMWareConfig().set_active_db("sqlite")

    #   for this example, we will use an embedding model that has been 'fine-tuned' for contracts
    embedding_model = "industry-bert-contracts"

    #   note: as of llmware==0.2.12, we have shifted from faiss to chromadb for the Fast Start examples
    #   --if you are using a Python version before 3.12, please feel free to substitute for "faiss"
    #   --for versions of Python >= 3.12, for the Fast Start examples (e.g., no install required), we
    #   recommend using chromadb or lancedb

    #   please double-check: `pip3 install chromadb` or pull the latest llmware version to get automatically
    #   -- if you have installed any other vector db, just change the name, e.g, "milvus" or "pg_vector"
    vector_db = "chromadb"

    # pick any name for the library
    lib_name = "example_5_library"
    example_models = ["llmware/bling-1b-0.1", "llmware/bling-tiny-llama-v0", "llmware/dragon-yi-6b-gguf"]

    # use local cpu model
    llm_model_name = example_models[0]

    #   to swap in a gpt-4 openai model - uncomment these two lines
    #   llm_model_name = "gpt-4"
    #   os.environ["USER_MANAGED_OPENAI_API_KEY"] = "<insert-your-openai-key>"
    semantic_rag(lib_name, embedding_model, llm_model_name)


update: Step 1 - Creating library: example_5_library
update: Step 2 - Downloading Sample Files
update: Step 3 - Parsing and Text Indexing Files

update: Step 4 - Generating Embeddings in chromadb db - with Model- industry-bert-contracts


C:\Users\hsyyu\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\hsyyu\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


update: embedding_handler - ChromaDB - Embeddings Created: 500 of 2211
update: embedding_handler - ChromaDB - Embeddings Created: 1000 of 2211
update: embedding_handler - ChromaDB - Embeddings Created: 1500 of 2211
update: embedding_handler - ChromaDB - Embeddings Created: 2000 of 2211
update: embedding_handler - ChromaDB - Embeddings Created: 2211 of 2211

update: Loading model for LLM inference -  llmware/bling-1b-0.1

Contract Name:  0 Amphitrite EXECUTIVE EMPLOYMENT AGREEMENT.pdf
Top Retrieval:  9 0.7264271974563599 a base salary at the annual rate of $5f00,000, payable semimonthly in   accordance with Employer's normal payroll practices. Executive's base salary shall be reviewed annually   by the Board (or the compensation committee of the Board), pursuant to Employer's normal   compensation and performance review policies for senior level executives, and may be increased but not   decreased. The amount of any 
Top Retrieval:  19 0.7347566485404968 increase for each year shall be 


update: llm answer -   $250,000.00

Contract Name:  7 Eileithyia EXECUTIVE EMPLOYMENT AGREEMENT.pdf
Top Retrieval:  3 0.7016128301620483 review policies for senior level executives, and may be increased but not   decreased. The amount of any increase for each year shall be determined accordingly. For purposes of this   Agreement, the term “Base Salary” shall mean the amount of Executive's base salary established from   time to time pursuant to this Section 2.2.    2.3. Incentive Bonus. Executive shall be entitled to participate 
Top Retrieval:  11 0.7294694185256958     (a) “Accrued Compensation” means all compensation, reimbursements and other amounts earned by,   payable to, or accrued and vested for Executive through and including Executive's Date of Termination,   but not paid as of Executive's Date of Termination, including, but not limited to, (i) Base Salary;  (ii) Executive's Incentive Bonus for the fiscal year that ended immediately prior to Executive's 
Top Retrieval:  26 0.


update: llm answer -   $250,000.00

Contract Name:  12 Nyx EXECUTIVE EMPLOYMENT AGREEMENT.pdf
Top Retrieval:  8 0.702351450920105 performance review policies for senior level executives, and may be increased but not   decreased. The amount of any increase for each year shall be determined accordingly. For purposes of this   Agreement, the term “Base Salary” shall mean the amount of Executive's base salary established from   time to time pursuant to this Section 2.2.    2.3. Incentive Bonus. Executive shall be entitled to 
Top Retrieval:  25 0.7486357688903809 Compensation” means all compensation, reimbursements and other amounts earned by,   payable to, or accrued and vested for Executive through and including Executive's Date of Termination,   but not paid as of Executive's Date of Termination, including, but not limited to, (i) Base Salary;  (ii) Executive's Incentive Bonus for the fiscal year that ended immediately prior to Executive's Date of   Termination, 
Top Retrieval:  40 0.7